In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/Tugas Akhir"

In [ ]:
import pandas as pd
import os

data = pd.read_csv(os.path.join(path, "preprocessed_data.csv"), encoding="latin-1")

In [ ]:
print("Shape dataset:", data.shape)
data.head()

Shape dataset: (12961, 10)


,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
0,saat semua cowok berusaha melacak perhatian gu...,1,1,1,0,0,0,0,0,1
1,siapa telat memberi tau kamu edan sarap gue be...,0,1,0,0,0,0,0,0,0
2,kadang aku berpikir aku tetap percaya tuhan pa...,0,0,0,0,0,0,0,0,0
3,aku aku ku tau matamu sipit dilihat mana aku,0,0,0,0,0,0,0,0,0
4,kaum cebong kafir kelihatan dongoknya awal tam...,1,1,0,1,1,0,0,0,0


#Inisialisasi Model Pra-Latih cahya/bert-base-indonesian-522M

In [ ]:
!pip install transformers tensorflow

In [ ]:
from transformers import BertTokenizer, TFBertModel

model_name = 'cahya/bert-base-indonesian-522M'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/545M [00:00<?, ?B/s]

Some layers from the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at cahya/bert-base-indonesian-522M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


#Menentukan jumlah max_length

In [ ]:
# Membaca data dan menyalin ke variabel baru
tokenized_data = data.copy()

# Konversi kolom 'Tweet' ke list of string
texts = tokenized_data['Tweet'].astype(str).tolist()

# Tokenisasi per kalimat, ambil tokens dan token IDs
tokens_list = []
token_ids_list = []

for text in texts:
    encoded = tokenizer(text)
    tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'])
    token_ids = encoded['input_ids']

    tokens_list.append(tokens)
    token_ids_list.append(token_ids)

# Tambahkan ke dataframe
tokenized_data['Tokens'] = tokens_list
tokenized_data['Token_IDs'] = token_ids_list

# Hapus kolom label agar fokus pada teks dan tokenisasi
tokenized_data = tokenized_data.drop(columns=['HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other'])

# Atur agar kolom tidak terpotong
pd.set_option('display.max_colwidth', None)

# Tampilkan 5 baris pertama
tokenized_data.head()


,Tweet,Tokens,Token_IDs
0,saat semua cowok berusaha melacak perhatian gue kamu lantas remehkan perhatian gue kasih khusus kamu basic kamu cowok bego,"[[CLS], saat, semua, cowok, berusaha, melacak, perhatian, gu, ##e, kamu, lantas, rem, ##ehkan, perhatian, gu, ##e, kasih, khusus, kamu, basic, kamu, cowok, beg, ##o, [SEP]]","[3, 1793, 2302, 24656, 4149, 16737, 4895, 2644, 1019, 8343, 10523, 4002, 14709, 4895, 2644, 1019, 6625, 2632, 8343, 19396, 8343, 24656, 4844, 1016, 1]"
1,siapa telat memberi tau kamu edan sarap gue bergaul cigax jifla calis sama siapa licew,"[[CLS], siapa, tel, ##at, memberi, tau, kamu, ed, ##an, sara, ##p, gu, ##e, bergaul, cig, ##ax, ji, ##fl, ##a, cali, ##s, sama, siapa, lic, ##ew, [SEP]]","[3, 6186, 2128, 1494, 3797, 8296, 8343, 2645, 1487, 5506, 1029, 2644, 1019, 23395, 19915, 25005, 2566, 9528, 1007, 21476, 1018, 2023, 6186, 12262, 1739, 1]"
2,kadang aku berpikir aku tetap percaya tuhan padahal aku selalu jatuh berkali kali kadang aku merasa tuhan meninggalkan aku sendirian orang tuaku berencana berpisah kakakku lebih memilih jadi kristen aku anak ter,"[[CLS], kadang, aku, berpikir, aku, tetap, percaya, tuhan, padahal, aku, selalu, jatuh, berkali, kali, kadang, aku, merasa, tuhan, meninggalkan, aku, sendirian, orang, tua, ##ku, berencana, berpisah, kakak, ##ku, lebih, memilih, jadi, kristen, aku, anak, ter, [SEP]]","[3, 3362, 4384, 7636, 4384, 1817, 4823, 4379, 9123, 4384, 3413, 3763, 13216, 1859, 3362, 4384, 4169, 4379, 3601, 4384, 12669, 1714, 3466, 4356, 8027, 11480, 6610, 4356, 1757, 3623, 3940, 3132, 4384, 1942, 1525, 1]"
3,aku aku ku tau matamu sipit dilihat mana aku,"[[CLS], aku, aku, ku, tau, mata, ##mu, sip, ##it, dilihat, mana, aku, [SEP]]","[3, 4384, 4384, 2030, 8296, 3122, 2608, 3908, 1531, 5032, 2369, 4384, 1]"
4,kaum cebong kafir kelihatan dongoknya awal tambah dungu haha,"[[CLS], kaum, ce, ##bong, kafir, kelihatan, dong, ##ok, ##nya, awal, tambah, dung, ##u, ha, ##ha, [SEP]]","[3, 3605, 12370, 12147, 19075, 19131, 6479, 1608, 1538, 2091, 18076, 28647, 1005, 3230, 2256, 1]"


In [ ]:
# Hitung panjang token per tweet
token_lengths = tokenized_data['Tokens'].apply(len)

# Statistik deskriptif
print("Statistik panjang token:")
print(token_lengths.describe())

# Panjang maksimum
max_len = token_lengths.max()
print(f"\nPanjang maksimum token: {max_len}")

# Kalau ingin lihat distribusi 95% quantile
percentile_95 = token_lengths.quantile(0.95)
print(f"Panjang token yang mencakup 95% data: {int(percentile_95)}")


Statistik panjang token:
count    12961.000000
mean        19.712291
std         10.489560
min          4.000000
25%         12.000000
50%         18.000000
75%         26.000000
max         85.000000
Name: Tokens, dtype: float64

Panjang maksimum token: 85
Panjang token yang mencakup 95% data: 40


In [ ]:
# Hitung panjang token dari setiap tweet
tokenized_data['Token_Length'] = tokenized_data['Tokens'].apply(len)

# Hitung panjang token 95 persen
percentile_95 = tokenized_data['Token_Length'].quantile(0.95)

# Filter baris yang panjang tokennya di atas percentile 95%
outlier_tokens = tokenized_data[tokenized_data['Token_Length'] > percentile_95]

# Tampilkan hasil
print(f"Jumlah tweet yang panjang tokennya di atas 95% data: {len(outlier_tokens)}")
outlier_tokens[['Tokens', 'Token_IDs', 'Token_Length']]


Jumlah tweet yang panjang tokennya di atas 95% data: 582


,Tokens,Token_IDs,Token_Length
8,"[[CLS], nah, admin, belanja, satu, port, terbaik, nak, makan, ais, kepa, ##l, milo, ais, kepa, ##l, hor, ##lic, ##ks, cend, ##ol, topi, ##ng, kau, kau, [UNK], a, dok, ##et, mana, gerai, ro, ##zak, mertua, ##ku, tai, ##pan, depan, kembar, baby, romantik, ##a, bank, islam, sena, ##wang, [UNK], [SEP]]","[3, 9336, 3703, 13376, 1723, 8234, 2911, 8746, 2464, 13494, 9285, 1011, 29706, 13494, 9285, 1011, 5268, 19263, 5212, 8241, 1547, 13267, 9563, 10900, 10900, 0, 40, 2960, 1583, 2369, 14288, 2705, 16519, 21843, 4356, 4412, 4621, 3358, 8821, 13363, 29013, 1007, 3634, 2638, 26672, 6598, 0, 1]",48
43,"[[CLS], gubernur, zaman, now, gus, ip, ##ul, put, ##i, ganja, ##r, yasin, dja, ##rot, sih, ##ar, hasan, anton, kos, ##ter, ace, nurdin, sudirman, karol, ##in, gi, ##dot, rt, keanekaragaman, budaya, suku, agama, sesungguhnya, kekayaan, [UNK], tutur, dja, ##rot, hadapan, peserta, rapat, kerja, cabang, khusus, [UNK], [SEP]]","[3, 3840, 3013, 6776, 6830, 5173, 1540, 1940, 1012, 24195, 1008, 27480, 25266, 10544, 16391, 1490, 7409, 6235, 3823, 1578, 19803, 25857, 14510, 15212, 1498, 4123, 28552, 6181, 15013, 3017, 2660, 2664, 8313, 7471, 0, 19547, 25266, 10544, 9448, 4417, 7492, 3287, 4173, 2632, 0, 1]",46
51,"[[CLS], aw, ##aa, ##ss, ##s, jangan, berita, ##nya, ke, saudara, ger, ##ung, susah, mencari, kosakata, ##nya, kalau, bocor, mem, ##ici, ##ng, mata, bibir, mon, ##yong, kes, ##amping, suara, keluar, ky, nh, ##n, kentu, ##t, politik, kotor, sekali, zaman, kalau, tahu, brt, pulang, ky, nh, ##n, nge, ##den, dung, ##u, [SEP]]","[3, 1907, 25457, 17541, 1018, 8492, 4432, 1538, 1519, 3747, 1992, 1563, 15822, 3759, 13879, 1538, 5356, 22677, 1546, 13883, 9563, 3122, 14541, 2746, 20456, 1828, 5205, 2881, 2201, 6027, 13322, 1010, 21477, 1009, 2605, 12092, 2779, 3013, 5356, 4796, 14957, 5509, 6027, 13322, 1010, 29965, 4841, 28647, 1005, 1]",50
55,"[[CLS], prov, ##okasi, mayat, politis, ##asi, agama, penyebab, kekalahan, pilihan, kepala, daerah, daerah, khusus, ibukota, begin, ##ilah, cara, ce, ##bong, mendeskripsikan, kekalahan, jun, ##jun, ##gan, ##nya, for, your, information, ahok, pernah, ikut, pemilihan, apapun, kecamatan, jadi, wakil, dia, bukanlah, etnis, mayoritas, [SEP]]","[3, 2266, 5151, 10966, 14064, 1558, 2664, 6488, 6702, 5311, 2548, 2042, 2042, 2632, 13437, 8886, 2486, 2870, 12370, 12147, 12930, 6702, 7209, 22565, 1884, 1538, 2923, 10339, 18403, 29708, 2295, 3821, 3501, 6546, 2168, 3940, 3487, 1831, 6765, 5531, 4813, 1]",42
58,"[[CLS], taka, ##bur, lahir, siapa, penguasa, rakyat, ulama, maupun, cend, ##iki, ##a, bahkan, iblis, makhluk, alim, beribadah, berat, ##us, ratus, tahun, dil, ##akna, ##t, has, ##ud, taka, ##burn, ##ya, hanya, seorang, ahli, ilmu, filsafat, tidak, fai, ##los, ##of, kita, demo, diri, sendi, [SEP]]","[3, 22994, 2278, 2444, 6186, 4087, 2977, 5646, 2754, 8241, 3518, 1007, 2921, 12894, 6054, 19574, 19225, 3739, 1521, 11668, 1582, 1851, 10129, 1009, 6298, 1575, 22994, 14774, 1508, 1996, 1783, 3677, 2739, 5996, 1653, 21714, 10726, 3097, 2557, 16102, 2436, 19542, 1]",43
...,...,...,...
12923,"[[CLS], lihat, fakta, dong, brow, ##sing, peringkat, partai, korup, juara, ##nya, partai, demokrasi, indonesia, perjuangan, kalau, ok, ##num, muslim, korupsi, ce, ##bong, gembira, ko, ##ar, ko, ##ar, kalau, ok, ##num, non, muslim, korupsi, ce, ##bong, diam, sebenarnya, kalian, ben, ##ch, [SEP]]","[3, 5927, 7439, 6479, 15200, 20808, 4007, 2618, 21527, 3871, 1538, 2618, 6784, 1766, 5494, 5356, 2275, 6644, 3924, 8535, 12370, 12147, 16320, 3040, 1490, 3040, 1490, 5356, 2275, 6644, 3712, 3924, 8535, 12370, 12147, 4403, 3876, 13581, 2416, 1731, 1]",41
12934,"[[CLS], bubar, ##kan, dewan, perwakilan, rakyat, bubar, ##kan, dewan, perwakilan, rakyat, bubar, ##kan, dewan, perwakilan, rakyat, bubar, ##kan, dewan, perwakilan, rakyat, bubar, ##kan, dewan, perwakilan, rakyat, bubar, ##kan, dewan, perwakilan, rakyat, bubar, ##kan, dewan, perwakilan, rakyat, bubar, ##kan, dewan, perwakilan, rakyat, [SEP]]","[3, 14170, 

In [ ]:
# Buat rangkuman jumlah teks untuk setiap panjang token
token_lengths = tokenized_data['Token_Length']
length_summary = token_lengths.value_counts().sort_index()

# Filter hanya panjang token yang berada di atas percentile 95
outlier_summary = length_summary[length_summary.index > percentile_95]

# Tampilkan hasil
print(outlier_summary)

Token_Length
41    92
42    63
43    65
44    58
45    54
46    37
47    35
48    27
49    22
50    21
51    18
52    16
53    19
54     6
55     9
56     5
57    12
58     6
59     3
60     2
62     3
63     1
64     1
66     3
69     1
70     1
79     1
85     1
Name: count, dtype: int64


#Tokenisasi dengan Pra-latih BERT

In [ ]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import time

batch_size = 32
texts = data['Tweet'].astype(str).tolist()

bert_embeddings = []

start = time.time()
for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
    batch_texts = texts[i:i+batch_size]
    encoded = tokenizer(batch_texts, return_tensors='tf', padding='max_length', truncation=True, max_length=40)
    output = model(encoded)

    bert_embeddings.append(output.last_hidden_state.numpy())
end = time.time()
print(f"Total time: {(end - start)/60:.2f} minutes")

Embedding: 100%|██████████| 406/406 [01:39<00:00,  4.10it/s]

Total time: 1.65 minutes


In [ ]:
final_embeddings = np.concatenate(bert_embeddings, axis=0)
np.save("bert_embedding.npy", final_embeddings)

In [ ]:
# Ambil salah satu teks asli
sample_text = texts[0]

# Tokenisasi
encoded = tokenizer(sample_text, return_tensors='tf', padding='max_length', truncation=True, max_length=40)
tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'][0])
token_ids = encoded['input_ids'][0].numpy()
attention_mask = encoded['attention_mask'][0].numpy()

# Proses ke model
output = model(encoded)
embedding = output.last_hidden_state[0].numpy()

# Tampilkan hasil
print("===== TEKS ASLI =====")
print(sample_text)

print("\n===== TOKENS =====")
token_lines = []
for i in range(0, len(tokens), 12):
    line = " ".join(f"{repr(tok):>10}" for tok in tokens[i:i+12])
    token_lines.append(line)
print("[")
for line in token_lines:
    print(f" {line}")
print("]")

print("\n===== TOKEN IDs =====")
print(token_ids)

print("\n===== ATTENTION MASK =====")
print(attention_mask)

print("\n===== SHAPE EMBEDDING =====")
print(embedding.shape)

print("\n===== VEKTOR EMBEDDING =====")
np.set_printoptions(suppress=True)
for i, vec in enumerate(embedding):
    print(f"Token ke-{i+1} ({token_ids[i]}): {vec}")


===== TEKS ASLI =====
saat semua cowok berusaha melacak perhatian gue kamu lantas remehkan perhatian gue kasih khusus kamu basic kamu cowok bego

===== TOKENS =====
[
    '[CLS]'     'saat'    'semua'    'cowok' 'berusaha'  'melacak' 'perhatian'       'gu'      '##e'     'kamu'   'lantas'      'rem'
  '##ehkan' 'perhatian'       'gu'      '##e'    'kasih'   'khusus'     'kamu'    'basic'     'kamu'    'cowok'      'beg'      '##o'
    '[SEP]'    '[PAD]'    '[PAD]'    '[PAD]'    '[PAD]'    '[PAD]'    '[PAD]'    '[PAD]'    '[PAD]'    '[PAD]'    '[PAD]'    '[PAD]'
    '[PAD]'    '[PAD]'    '[PAD]'    '[PAD]'
]

===== TOKEN IDs =====
[    3  1793  2302 24656  4149 16737  4895  2644  1019  8343 10523  4002
 14709  4895  2644  1019  6625  2632  8343 19396  8343 24656  4844  1016
     1     2     2     2     2     2     2     2     2     2     2     2
     2     2     2     2]

===== ATTENTION MASK =====
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]

=====

In [ ]:
# salin bert_embedding.npy ke google drive
import shutil

shutil.copy("bert_embedding.npy", "/content/drive/MyDrive/Colab Notebooks/Tugas Akhir/bert_embedding.npy")

'/content/drive/MyDrive/Colab Notebooks/Tugas Akhir/bert_embedding.npy'